# Overview
We are creating a Streamlit web app to run on Raspberry Pi. The app should do object detection using our YOLOv8 model in ONNX format.

Code is based off https://github.com/JackDance/YOLOv8-streamlit-app

# Download Model

In [1]:
!gdown --id 1hvpitHtXvmUZhLa7ebfTxrfXn186VrF3

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1hvpitHtXvmUZhLa7ebfTxrfXn186VrF3
To: /content/Vending-YOLOv8n.onnx
100% 12.2M/12.2M [00:00<00:00, 27.2MB/s]


# Packages

In [2]:
!pip install ultralytics -q
!pip install streamlit -q
!pip install onnxruntime -q
!pip install onnx -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.1/800.1 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 46.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 63.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.2 MB/s eta 0:00:00


# Streamlit App

In [9]:
%%writefile app.py

from ultralytics import YOLO
import streamlit as st
import cv2
from PIL import Image
import tempfile

def _display_detected_frames(conf, model, st_frame, image):
    """
    Display the detected objects on a video frame using the YOLOv8 model.
    :param conf (float): Confidence threshold for object detection.
    :param model (YOLOv8): An instance of the `YOLOv8` class containing the YOLOv8 model.
    :param st_frame (Streamlit object): A Streamlit object to display the detected video.
    :param image (numpy array): A numpy array representing the video frame.
    :return: None
    """
    # Resize the image to a standard size
    image = cv2.resize(image, (720, int(720 * (9 / 16))))

    # Predict the objects in the image using YOLOv8 model
    res = model.predict(image, conf=conf)

    # Plot the detected objects on the video frame
    res_plotted = res[0].plot()
    st_frame.image(res_plotted,
                   caption='Detected Video',
                   channels="BGR",
                   use_column_width=True
                   )
    num_brown = 0
    num_red = 0
    for pred in res[0].boxes.cls:
        if pred == 0:
            num_brown += 1
        elif pred == 1:
            num_red += 1

    st.subheader(f"Number of brown: {num_brown}")
    st.subheader(f"Number of red: {num_red}")


@st.cache_resource
def load_model(model_path):
    """
    Loads a YOLO object detection model from the specified model_path.

    Parameters:
        model_path (str): The path to the YOLO model file.

    Returns:
        A YOLO object detection model.
    """
    model = YOLO(model_path)
    return model


def infer_uploaded_image(conf, model):
    """
    Execute inference for uploaded image
    :param conf: Confidence of YOLOv8 model
    :param model: An instance of the `YOLOv8` class containing the YOLOv8 model.
    :return: None
    """
    source_img = st.sidebar.file_uploader(
        label="Choose an image...",
        type=("jpg", "jpeg", "png", 'bmp', 'webp')
    )

    col1, col2 = st.columns(2)

    with col1:
        if source_img:
            uploaded_image = Image.open(source_img)
            # adding the uploaded image to the page with caption
            st.image(
                image=source_img,
                caption="Uploaded Image",
                use_column_width=True
            )

    if source_img:
        if st.button("Execution"):
            with st.spinner("Running..."):
                res = model.predict(uploaded_image,
                                    conf=conf)
                boxes = res[0].boxes
                res_plotted = res[0].plot()[:, :, ::-1]

                num_brown = 0
                num_red = 0
                for pred in res[0].boxes.cls:
                    if pred == 0:
                        num_brown += 1
                    elif pred == 1:
                        num_red += 1

                with col2:
                    st.image(res_plotted,
                             caption="Detected Image",
                             use_column_width=True)
                    st.subheader(f"Number of brown: {num_brown}")
                    st.subheader(f"Number of red: {num_red}")


def infer_uploaded_video(conf, model):
    """
    Execute inference for uploaded video
    :param conf: Confidence of YOLOv8 model
    :param model: An instance of the `YOLOv8` class containing the YOLOv8 model.
    :return: None
    """
    source_video = st.sidebar.file_uploader(
        label="Choose a video..."
    )

    if source_video:
        st.video(source_video)

    if source_video:
        if st.button("Execution"):
            with st.spinner("Running..."):
                try:
                    tfile = tempfile.NamedTemporaryFile()
                    tfile.write(source_video.read())
                    vid_cap = cv2.VideoCapture(
                        tfile.name)
                    st_frame = st.empty()
                    while (vid_cap.isOpened()):
                        success, image = vid_cap.read()
                        if success:
                            _display_detected_frames(conf,
                                                     model,
                                                     st_frame,
                                                     image
                                                     )
                        else:
                            vid_cap.release()
                            break
                except Exception as e:
                    st.error(f"Error loading video: {e}")


def infer_uploaded_webcam(conf, model):
    """
    Execute inference for webcam.
    :param conf: Confidence of YOLOv8 model
    :param model: An instance of the `YOLOv8` class containing the YOLOv8 model.
    :return: None
    """
    try:
        flag = st.button(
            label="Stop running"
        )
        vid_cap = cv2.VideoCapture(0)  # local camera
        st_frame = st.empty()
        while not flag:
            success, image = vid_cap.read()
            if success:
                _display_detected_frames(
                    conf,
                    model,
                    st_frame,
                    image
                )
            else:
                vid_cap.release()
                break
    except Exception as e:
        st.error(f"Error loading video: {str(e)}")

def main():
    #!/usr/bin/env python
    # -*- coding: utf-8 -*-
    from pathlib import Path
    from PIL import Image
    import streamlit as st


    # setting page layout
    st.set_page_config(
        page_title="Interactive Interface for YOLOv8",
        page_icon="🤖",
        layout="wide",
        initial_sidebar_state="expanded"
        )

    # main page heading
    st.title("Interactive Interface for YOLOv8")

    # load pretrained DL model
    model = load_model('/content/Vending-YOLOv8n.onnx')

    # image/video options
    st.sidebar.header("Image/Video Config")
    source_selectbox = st.sidebar.selectbox(
        "Select Source",
        ["Image", "Video", "Webcam"]
    )

    source_img = None
    confidence = 0.5
    if source_selectbox == "Image": # Image
        infer_uploaded_image(confidence, model)
    elif source_selectbox == "Video": # Video
        infer_uploaded_video(confidence, model)
    elif source_selectbox == "Webcam": # Webcam
        infer_uploaded_webcam(confidence, model)
    else:
        st.error("Currently only 'Image' and 'Video' source are implemented")

if __name__ == "__main__":
    main()

Overwriting app.py


In [5]:
!curl ipecho.net/plain
# THIS IS THE PASSWORD FOR LOCALTUNNEL

34.91.23.244

In [ ]:
!streamlit run app.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.91.23.244:8501

npx: installed 22 in 2.347s
your url is: https://afraid-poets-obey.loca.lt
WARNING ⚠️ Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify','pose' or 'obb'.
Loading /content/Vending-YOLOv8n.onnx for ONNX Runtime inference...

0: 640x640 10 Browns, 11 Reds, 240.9ms
Speed: 6.4ms preprocess, 240.9ms inference, 2.1ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 10 Browns, 10 Reds, 356.2ms
Speed: 6.8ms preprocess, 356.2ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 10 Browns, 10 Reds, 295.9ms
Speed: 4.9ms preprocess, 295.9ms inference, 4.1ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 10 Browns, 10 Reds, 279.9ms
Speed: 5.0ms preprocess, 279.9ms inference, 3.7ms pos